<a href="https://colab.research.google.com/github/MatthewYancey/16-9GAN/blob/master/src/process_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

This notebook takes the video files in a folder and saves the individual frames.

## Imports and Parameters

In [1]:
import os
import glob
import cv2
import shutil
from google.colab import drive
import zipfile

In [2]:
# parameters
drive.mount('/content/gdrive')
VIDEO_PATH = '/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 16:9/'
TEST_PATH = '/content/gdrive/My Drive/16:9GAN/data_raw/FMA - 4:3/'
FRAME_PATH = '/content/frames/'
ZIP_FOLDER = '/content/gdrive/My Drive/16:9GAN/data_out/'

FRAME_SECONDS_SKIP = 0.5
IMAGE_HEIGHT = 256
SINGLE_SIDE = 32
# IMAGE_WIDTH = 456
IMAGE_WIDTH = 256
SKIP_SECONDS_BEGINNING = 120
SKIP_SECONDS_END = 120
TEST_SPLIT = 0.25

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Video loop and frame saving

In [ ]:
def save_frames(video_list, zip_path, test_video=False):

    try:
        shutil.rmtree(FRAME_PATH)
    except FileNotFoundError:
        pass
    
    os.mkdir(FRAME_PATH)

    frame_count = 0
    for f in video_list:
        print(f)
        vidcap = cv2.VideoCapture(f)
        video_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT) / 24)
        success, image = vidcap.read()

        # loops through and save the frames
        while success:
            current_frame = vidcap.get(cv2.CAP_PROP_POS_FRAMES)

            # skips the intro, outtro, and every 24 frames
            if test_video:
                if current_frame % (FRAME_SECONDS_SKIP * 24) == 0:                    
                    # resizes and save the frame
                    image = cv2.resize(image, (IMAGE_WIDTH - (2 * SINGLE_SIDE), IMAGE_HEIGHT))
                    cv2.imwrite(f'{FRAME_PATH}{frame_count}.jpg', image)
                    frame_count += 1

            else:
                if (current_frame >= (SKIP_SECONDS_BEGINNING * 24) and current_frame <= (video_length - SKIP_SECONDS_END) * 24 and current_frame % (FRAME_SECONDS_SKIP * 24) == 0):
                    # resizes and save the frame
                    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
                    cv2.imwrite(f'{FRAME_PATH}{frame_count}.jpg', image)
                    frame_count += 1

            # loop to the next frame
            success, image = vidcap.read()
                
        print(f'Number of images saved: {frame_count}')

    shutil.make_archive(zip_path, 'zip', FRAME_PATH)
    print(f'Saved zip {zip_path}')

# keeps one video out for testing
video_files = glob.glob(VIDEO_PATH + '*')
video_files.sort()
train_files = video_files[int(len(video_files) * TEST_SPLIT):]
val_files = video_files[:int(len(video_files) * TEST_SPLIT)]
test_files = glob.glob(TEST_PATH + '*')

print(f'Train videos: {len(train_files)}')
print(f'Validate videos: {len(val_files)}')
print(f'Test videos: {len(test_files)}')

# saves a train an test zip file
save_frames(test_files, ZIP_FOLDER + 'test', True)
save_frames(val_files, ZIP_FOLDER + 'validate')
save_frames(train_files, ZIP_FOLDER + 'train')
